In [1]:
from jetbot import Robot
import sys
import glob
import serial
import time
import datetime

In [2]:
def serial_ports():

    """ Lists serial port names
        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """

    ports = glob.glob('/dev/ttyUSB*') 
    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result

In [3]:
def read_sensor():
    logfile = 'DGS_Log_Dec_29_2020'
    device_identfier = ','
    output_response_time = 1
    baudrate = 9600
    serialport = ['','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    serialdata = ['','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    serial_port_list = []
    device_list = []
    port_list = serial_ports()
    i = 0
    while(i < len(port_list)):	
        serialport[i] =  serial.Serial(port_list[i], baudrate, timeout=5)
        i = i + 1
        #time.sleep(.5)#1
        
        i = 0
        while(i < len(port_list)):
            #print("port_list length: {}".format(len(port_list)))
            serialport[i].write('\r'.encode())
            #time.sleep(.5)#5 increase this if we add another sensor i think
            serialport[i].write('\r'.encode()) 
            serialdata[i]  = serialport[i].readline()
            time.sleep(output_response_time)
            i = i + 1
    i = 0
    while(i < len(port_list)):
        try:
            if device_identfier.encode() in serialdata[i]:
                serial_port_list.append(port_list[i])
                device_list.append(serialdata[i].decode('utf-8')[:12])
        except:

            pass

        i = i + 1 
    i = 0

    while(i < len(port_list)):
        serialport[i].close()
        i = i + 1
    #print(device_list)
    serial_port_list.sort(key=dict(zip(serial_port_list, device_list)).get)
    #print(serial_port_list)
    
    i = 0
    while(i < len(serial_port_list)):
        sensor_data = serialdata[i].decode('utf-8')[:-3]   #SAMPLE: 121216010434, 63, 21, 33, 32610, 25588, 20602, 00, 00, 12, 45
        data_array = [x.strip() for x in sensor_data.split(',')]
        #print(sensor_data)
        O3 = int(data_array[1])
        print("O3 level: {}".format(O3))
        i = i + 1
    i = 0 
    
    # close all open serial ports
    while(i < len(serial_port_list)):
        serialport[i].close()
        i = i + 1
        
    return O3
    

In [4]:
def read_sensor_continuous():
    logfile = 'DGS_Log_Dec_29_2020'
    device_identfier = ','
    output_response_time = 1
    baudrate = 9600
    serialport = ['','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    serialdata = ['','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    serial_port_list = []
    device_list = []


##################Ordering the serial ports based on device name##################
    clock = str(datetime.datetime.now())
    print("Gathering ports list at " + clock)
    port_list = serial_ports()
    #print(port_list)
    
    i = 0
    while(i < len(port_list)):	
        serialport[i] =  serial.Serial(port_list[i], baudrate, timeout=5)
        i = i + 1
        #time.sleep(.5)#1
        
        i = 0
        while(i < len(port_list)):
            print("port_list length: {}".format(len(port_list)))
            serialport[i].write('\r'.encode())
            #time.sleep(.5)#5 increase this if we add another sensor i think
            serialport[i].write('\r'.encode()) 
            serialdata[i]  = serialport[i].readline()
            time.sleep(output_response_time)
            i = i + 1
    i = 0
    while(i < len(port_list)):
        try:
            if device_identfier.encode() in serialdata[i]:
                serial_port_list.append(port_list[i])
                device_list.append(serialdata[i].decode('utf-8')[:12])
        except:

            pass

        i = i + 1 
    i = 0

    while(i < len(port_list)):
        serialport[i].close()
        i = i + 1
    print(device_list)
    serial_port_list.sort(key=dict(zip(serial_port_list, device_list)).get)
    print(serial_port_list)

##########################Serial ports opened in the new order and output read###########################

    i = 0

    while(i < len(serial_port_list)):
        serialport[i] =  serial.Serial(serial_port_list[i], baudrate, timeout=0)
        i = i + 1

    #time.sleep(1) #2
    clock = str(datetime.datetime.now())
    print("Opening file for writing at " + clock)
    
    while 1:
        file = open(logfile, 'a') # append mode
        j = 0
        while(j < 12):
            i = 0
            while(i < len(serial_port_list)):
                serialport[i].write('\r'.encode())
                time.sleep(0.01)
                i = i + 1
            time.sleep(1) #5 This is how often to read from sensor (min is 1 second)
            i = 0
            while(i < len(serial_port_list)):
                serialdata[i]  = serialport[i].readline()
                i = i + 1
            clock = str(datetime.datetime.now())
            if device_identfier.encode() in serialdata[0]:
                file.write(clock + ', ')
                i = 0
                while(i < len(serial_port_list)):
                    file.write(serialdata[i].decode('utf-8')[:-3] + ', ')
                    sensor_data = serialdata[i].decode('utf-8')[:-3]   #SAMPLE: 121216010434, 63, 21, 33, 32610, 25588, 20602, 00, 00, 12, 45
                    data_array = [x.strip() for x in sensor_data.split(',')]
                    #print(sensor_data)
                    O3 = int(data_array[1])
                    print("O3 level: {}".format(O3))
                    i = i + 1

                file.write('\n')
                # time.sleep(55)
                j = j + 1
        print("Logging data: " + logfile + ' ' + clock)

        file.close()

    i = 0 
    
    # close all open serial ports
    while(i < len(serial_port_list)):
        serialport[i].close()
        i = i + 1
#########################################################################################################
    """
    The format of the sensor_data output is:
    SN, PPB, T (°C), RH (%), ADC Raw, T Raw, RH Raw, Day, Hour, Minute, Second
    """

    return 0

In [5]:
def read_sensor_finite(reads):
    logfile = 'DGS_Log'
    device_identfier = ','
    output_response_time = 1
    baudrate = 9600
    serialport = ['','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    serialdata = ['','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    serial_port_list = []
    device_list = []


##################Ordering the serial ports based on device name##################
    clock = str(datetime.datetime.now())
    print("Gathering ports list at " + clock)
    port_list = serial_ports()
    #print(port_list)
    
    i = 0
    while(i < len(port_list)):	
        serialport[i] =  serial.Serial(port_list[i], baudrate, timeout=5)
        i = i + 1
        #time.sleep(.5)#1
        
        i = 0
        while(i < len(port_list)):
            print("port_list length: {}".format(len(port_list)))
            serialport[i].write('\r'.encode())
            #time.sleep(.5)#5 increase this if we add another sensor i think
            serialport[i].write('\r'.encode()) 
            serialdata[i]  = serialport[i].readline()
            time.sleep(output_response_time)
            i = i + 1
    i = 0
    while(i < len(port_list)):
        try:
            if device_identfier.encode() in serialdata[i]:
                serial_port_list.append(port_list[i])
                device_list.append(serialdata[i].decode('utf-8')[:12])
        except:

            pass

        i = i + 1 
    i = 0

    while(i < len(port_list)):
        serialport[i].close()
        i = i + 1
    print(device_list)
    serial_port_list.sort(key=dict(zip(serial_port_list, device_list)).get)
    print(serial_port_list)

##########################Serial ports opened in the new order and output read###########################

    i = 0

    while(i < len(serial_port_list)):
        serialport[i] =  serial.Serial(serial_port_list[i], baudrate, timeout=0)
        i = i + 1

    #time.sleep(1) #2
    clock = str(datetime.datetime.now())
    print("Opening file for writing at " + clock)
    O3_sum = 0
    index = 0
    file = open(logfile, 'a') # append mode
    j = 0
    while(j < reads):
        i = 0
        while(i < len(serial_port_list)):
            serialport[i].write('\r'.encode())
            time.sleep(0.01)
            i = i + 1
        time.sleep(1) #5 This is how often to read from sensor (min is 1 second)
        i = 0
        while(i < len(serial_port_list)):
            serialdata[i]  = serialport[i].readline()
            i = i + 1
        clock = str(datetime.datetime.now())
        if device_identfier.encode() in serialdata[0]:
            file.write(clock + ', ')
            i = 0
            while(i < len(serial_port_list)):
                file.write(serialdata[i].decode('utf-8')[:-3] + ', ')
                sensor_data = serialdata[i].decode('utf-8')[:-3]   #SAMPLE: 121216010434, 63, 21, 33, 32610, 25588, 20602, 00, 00, 12, 45
                data_array = [x.strip() for x in sensor_data.split(',')]
                #print(sensor_data)
                O3 = int(data_array[1])
                O3_sum = O3_sum + O3
                print("O3 level: {}".format(O3))
                i = i + 1

            file.write('\n')
            # time.sleep(55)
            j = j + 1
    print("Logging data: " + logfile + ' ' + clock)

    file.close()

    i = 0 
    
    # close all open serial ports
    while(i < len(serial_port_list)):
        serialport[i].close()
        i = i + 1
#########################################################################################################
    """
    The format of the sensor_data output is:
    SN, PPB, T (°C), RH (%), ADC Raw, T Raw, RH Raw, Day, Hour, Minute, Second
    """

    return O3_sum / reads

In [6]:
#read_sensor_finite(256)

In [7]:
#read_sensor_continuous()

In [8]:
start = time.time()
print("test")
end = time.time()
print(end - start)

test
0.000568389892578125


In [9]:
robot = Robot()
robot.stop()

In [10]:
def square_routine():
    turn_x = -0.3
    turn_y = 0.3

    forward_x = 0.325
    forward_y = 0.3

    pause = 0.5

    position_1 = 0
    position_2 = 0
    position_3 = 0
    position_4 = 0
    
    ####################################
    ######## SQUARE ROUTINE ############
    position_1 = read_sensor()
    robot.set_motors(turn_x, turn_y)
    time.sleep(.45)
    robot.stop()
    time.sleep(pause)
    robot.set_motors(forward_x, forward_y)
    time.sleep(1.0)
    robot.stop()
    time.sleep(pause)
    position_2 = read_sensor()
    robot.set_motors(turn_x, turn_y)
    time.sleep(.45)
    robot.stop()
    time.sleep(pause)
    robot.set_motors(forward_x, forward_y)
    time.sleep(1.0)
    robot.stop()
    time.sleep(pause)
    position_3 = read_sensor()
    robot.set_motors(turn_x, turn_y)
    time.sleep(.45)
    robot.stop()
    time.sleep(pause)
    robot.set_motors(forward_x, forward_y)
    time.sleep(1.0)
    robot.stop()
    time.sleep(pause)
    position_4 = read_sensor()
    robot.set_motors(turn_x, turn_y)
    time.sleep(.45)
    robot.stop()
    time.sleep(pause)
    robot.set_motors(forward_x, forward_y)
    time.sleep(1.0)
    robot.stop()
    time.sleep(pause)
    robot.stop()
    
    high = (max(position_1, position_2, position_3, position_4))
    if position_1 == high:
        return 1
    elif position_2 == high:
        return 2
    elif position_3 == high:
        return 3
    elif position_1 == high:
        return 4

In [11]:
def line_routine(reads):
    turn_x = -0.3
    turn_y = 0.3

    forward_x = 0.325
    forward_y = 0.3

    pause = 0.5

    position_1 = 0
    position_2 = 0
    
    ####################################
    ########## LINE ROUTINE ############
    position_1 = read_sensor_finite(reads)
    robot.set_motors(forward_x, forward_y)
    time.sleep(1.0)
    robot.stop()
    position_2 = read_sensor_finite(reads)

    
    high = (max(position_1, position_2))
    if position_1 == high:
        return 1
    else:
        return 2

In [12]:
def move_forward():
    forward_x = 0.325
    forward_y = 0.3
    robot.set_motors(forward_x, forward_y)
    time.sleep(1.0)
    robot.stop()
    
def move_back():
    forward_x = -0.325
    forward_y = -0.3
    robot.set_motors(forward_x, forward_y)
    time.sleep(1.0)
    robot.stop()

def turn_and_move():
    turn_x = -0.3
    turn_y = 0.3
    robot.set_motors(turn_x, turn_y)
    time.sleep(.45)
    robot.stop()
    robot.set_motors(forward_x, forward_y)
    time.sleep(1.0)
    robot.stop()
    

In [ ]:
# THIS RUNS SQUARE ROUTINE

while 1:
    turn_x = -0.3
    turn_y = 0.3

    forward_x = 0.325
    forward_y = 0.3

    pause = 0.5
    max_O3 = square_routine()

    if max_O3 == 1:
        move_forward()
    elif max_O3 == 2:
        turn_and_move()
        move_forward()
    elif max_O3 == 3:
        turn_and_move()
        turn_and_move()
        move_forward()
    elif max_O3 == 1:
        turn_and_move()
        turn_and_move()
        turn_and_move()
        move_forward()
    

In [14]:
# THIS RUNS LINE ROUTINE

while 1:
    turn_x = -0.3
    turn_y = 0.3

    forward_x = 0.325
    forward_y = 0.3

    pause = 0.5
    max_O3 = line_routine(15)

    if max_O3 == 1:
        move_forward()
    elif max_O3 == 2:
        move_back()
        move_back()

Gathering ports list at 2021-01-15 01:00:33.154783
port_list length: 1
['121216010434']
['/dev/ttyUSB0']
Opening file for writing at 2021-01-15 01:00:34.865583
O3 level: 7
O3 level: 14
O3 level: 14
O3 level: 15
O3 level: 24
O3 level: 21
O3 level: 27
O3 level: 19
O3 level: 16
O3 level: 18
O3 level: 16
O3 level: 19
O3 level: 19
O3 level: 17
O3 level: 11
Logging data: DGS_Log 2021-01-15 01:00:50.269737
Gathering ports list at 2021-01-15 01:00:51.326826
port_list length: 1
['121216010434']
['/dev/ttyUSB0']
Opening file for writing at 2021-01-15 01:00:52.539672
O3 level: 13
O3 level: 18
O3 level: 12
O3 level: 18
O3 level: 6
O3 level: 2
O3 level: 9
O3 level: 8
O3 level: 10
O3 level: 10
O3 level: 15
O3 level: 11
O3 level: 13
O3 level: 11
O3 level: 13
Logging data: DGS_Log 2021-01-15 01:01:07.954772
Gathering ports list at 2021-01-15 01:01:09.022395
port_list length: 1
['121216010434']
['/dev/ttyUSB0']
Opening file for writing at 2021-01-15 01:01:10.205883
O3 level: 13
O3 level: 10
O3 level: 1

KeyboardInterrupt: 

In [17]:
### OZONE DETECTION BY 2 FT ###

while 1:
    threshold = 255
    ozone_read = read_sensor_finite(15)
    print("Mean ozone level: {}".format(ozone_read))
    if ozone_read > threshold:
        print("OZONE DETECTED WITHIN AREA")
        break
    else:
        move_forward()

Gathering ports list at 2021-02-02 04:20:17.692742
port_list length: 1
['121216010434']
['/dev/ttyUSB0']
Opening file for writing at 2021-02-02 04:20:19.147686
O3 level: 110
O3 level: 112
O3 level: 111
O3 level: 109
O3 level: 108
O3 level: 111
O3 level: 108
O3 level: 103
O3 level: 107
O3 level: 107
O3 level: 107
O3 level: 106
O3 level: 107
O3 level: 109
O3 level: 104
Logging data: DGS_Log 2021-02-02 04:20:34.535577
Mean ozone level: 107.93333333333334
Gathering ports list at 2021-02-02 04:20:35.595056
port_list length: 1
['121216010434']
['/dev/ttyUSB0']
Opening file for writing at 2021-02-02 04:20:36.860109
O3 level: 103
O3 level: 107
O3 level: 109
O3 level: 109
O3 level: 110
O3 level: 103
O3 level: 107
O3 level: 103
O3 level: 106
O3 level: 105
O3 level: 104
O3 level: 103
O3 level: 111
O3 level: 114
O3 level: 116
Logging data: DGS_Log 2021-02-02 04:20:52.246943
Mean ozone level: 107.33333333333333
Gathering ports list at 2021-02-02 04:20:53.309891
port_list length: 1
['121216010434']
